In [ ]:
# Copyright 2024 Stephan Bscheider sbsch@bu.edu
# Copyright 2024 Humzah Durrani hhd8@bu.edu
# Copyright 2024 Alex Tianji Sun tianjis@bu.edu

Lesson on various mechanical principles of motion

1. Four Bar Linkage 
2. Cam Follower
3. Gear Ratio 

In [ ]:
## Four Bar Linkage

Cam Follower
    
A cam follower is one type mechanical device that is used to translate rotational motion into linear motion. The rotating element or the cam usually has a distict profile or shape, and as it rotates the follower, which is stationary in the x axis, follows the profile and "tracks" the y position of the rotating surface. 

This function is distict from something like a piston, that translates linear motion to rotational motion, because the cam profile can be adjusted to get specific period behavior from the the linear follower. In the following example the cam is half circle and half ellipse, so for 180 degrees of the rotation, or half the time, the follower will stay still, then for the other 180 degrees of rotation the follower will trace a parabolic curve based on the size of the ellipse. In the model try adjusting the rotation speed, major and minor axes lengths and see how the vertical position plot reacts.

In [1]:
## Cam Follower Code
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg') #This works nice using tk backend to run the matplotlib window so it can update out of line with the notebook

cv.destroyAllWindows()

## Initialize Variables for CV animation

center = (256, 256)  # Center of the oval
stick_height = 50  # Height of the stick
stick_pos_x = 256  # X position of the stick
i=0
angle = 0
majorAxis = 75  # Length of the axes of the oval
minorAxis = 50  
angle_step = 2  # Angle step for rotation

## Initialize OPEN CV Animation
# Create a window to display the animation
cv.namedWindow('Cam Follower', cv.WINDOW_NORMAL)
cv.resizeWindow('Cam Follower', 800, 800)

# Placeholder empty funct for trackbars
def nothing(x):
    pass

# Create trackbars for adjusting cam speed and axes
cv.createTrackbar('Speed', 'Cam Follower', 2, 10, nothing)
cv.createTrackbar('Major Axis', 'Cam Follower', 75, 200, nothing)
cv.createTrackbar('Minor Axis', 'Cam Follower', 50, 200, nothing)


## Initialize Matplotlib
t = np.linspace(1,100,100)
ytemp = np.zeros(100) # Array for plotting 

# Create figure and axes
plt.ion()
fig, ax = plt.subplots()
line1, = ax.plot(t, ytemp, label='Follower Height')
ax.legend()

# Set up plot limits
ax.set_xlim(0, 100)
ax.set_ylim(0,200)


while True:
    # Create a black image
    img = np.zeros((512, 512, 3), np.uint8)
    
    # Calculate the rotation angle every loop
    angle = i * angle_step
    
    # Create vector of points representing the cam for calculation use later
    Cvector = cv.ellipse2Poly(center, (minorAxis,minorAxis), angle, 0, 360, 1)
    Evector = cv.ellipse2Poly(center, (majorAxis,minorAxis), angle, 0, 360, 1)
    
    # Draw ellipses to represent the calculated cam
    cv.ellipse(img, center, (majorAxis,minorAxis), angle, 90, 270, (255, 0, 0), -1)
    cv.ellipse(img, center, (minorAxis,minorAxis), angle, -90, 90, (255, 0, 0), -1)
    cv.ellipse(img, center, (6,6), 0, 0, 360, (10, 10, 10), -1)
    
    # depending on the angle (on circle side or ellipse side of cam), calcuate the largest y value for a specifc x value
    if np.sin(np.radians(angle)) >= 0:
        matching = np.where((Evector[:,0] >= stick_pos_x-2) & (Evector[:,0] <= stick_pos_x+2))[0]
        yheight = min(Evector[matching[:],1])
    elif np.sin(np.radians(angle)) < 0:
        matching = np.where((Cvector[:,0] >= stick_pos_x-2) & (Cvector[:,0] <= stick_pos_x+2))[0]
        yheight = min(Cvector[matching[:],1])
    else:
        pass
    
    #Draw Follower Stick
    cv.line(img,(stick_pos_x,yheight-stick_height),(stick_pos_x,yheight),(0,255,0),2)
    
    # Display the image
    cv.imshow('Cam Follower', img)

    # Matplotlib update, Shift the data in the ytemp over one and add the new data point
    ytemp = np.append(ytemp, -yheight + center[1])
    ytemp = ytemp[1:]
    line1.set_xdata(t)  # Update the x data
    line1.set_ydata(ytemp)  # Update the y data
    fig.canvas.draw_idle()  # Plot updated figure
    fig.canvas.flush_events()  # This does somethign to not crash the code
  

    # Recheck the trackbar values at the end of each loop and reassign them to the vars
    angle_step = cv.getTrackbarPos('Speed', 'Cam Follower')
    majorAxis = cv.getTrackbarPos('Major Axis', 'Cam Follower')
    minorAxis = cv.getTrackbarPos('Minor Axis', 'Cam Follower')
    
    # Increment for the angle
    i = i + 1

    # This is the exit for the loop 'q' to quit
    if cv.waitKey(50) & 0xFF == ord('q'):
        break

# Destroy the window after quit
cv.destroyAllWindows()
plt.close(fig)


In [ ]:
## Gear Ratio